In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
########### c'est cette version qu'il faut installer car toute les version que j'installe ici ne marche pas j;ai pris celle qui marche sur mon pc
########## Ne pas executer deux fois #############

!pip install --upgrade tensorflow==2.12.0
!pip install transformers==4.37.1

In [ ]:
########### c'est cette version qu'il faut installer car toute les version que j'installe ici ne marche pas j;ai pris celle qui marche sur mon pc


import tensorflow as tf
import transformers

print("Version de TensorFlow :", tf.__version__)
print("Version de Transformers :", transformers.__version__)

In [ ]:
################ Bon Code ###############
#### charger et lire un dataset [.zip] sur coolab ####

import zipfile
import os
import pandas as pd

import shutil


chemin_zip_heavy_attacks = "AttacksHeavy.zip"
chemin_zip_heavy_benign = "BenignHeavy.zip"


chemin_zip_light_attacks = "AttacksLight.zip"
chemin_zip_light_benign = "BenignLight.zip"


# Fonction pour extraire les fichiers zip
def extraire_zip(chemin_zip):
    with zipfile.ZipFile(chemin_zip, 'r') as zip_ref:
        zip_ref.extractall("extraction_temp")  # Extraire les fichiers zip dans un répertoire temporaire

# Fonction pour charger les fichiers CSV d'un type spécifique (stateful ou stateless)
def charger_concatener_donnees(sous_dossier, prefixe):
    # Lister tous les fichiers CSV dans le sous-dossier
    fichiers_csv = [f for f in os.listdir(f"extraction_temp/{sous_dossier}") if f.startswith(prefixe) and f.endswith('.csv')]
    # Lire chaque fichier CSV et le stocker dans une liste de DataFrames
    dataframes = [pd.read_csv(f"extraction_temp/{sous_dossier}/{f}") for f in fichiers_csv]
    # Concaténer les DataFrames en un seul
    return pd.concat(dataframes, ignore_index=True)

# Extraire les fichiers zip
extraire_zip(chemin_zip_heavy_attacks)
extraire_zip(chemin_zip_heavy_benign)

extraire_zip(chemin_zip_light_attacks)
extraire_zip(chemin_zip_light_benign)

########## Heavy ############

# Charger et concaténer les données stateful
stateful_heavy_attack_data = charger_concatener_donnees("AttacksHeavy", "stateful")
stateful_heavy_benign_data = charger_concatener_donnees("BenignHeavy", "stateful")

# Charger et concaténer les données stateless
stateless_heavy_attack_data = charger_concatener_donnees("AttacksHeavy", "stateless")
stateless_heavy_benign_data = charger_concatener_donnees("BenignHeavy", "stateless")


#### Light ###############

# Charger et concaténer les données stateful
stateful_light_attack_data = charger_concatener_donnees("AttacksLight", "stateful")
stateful_light_benign_data = charger_concatener_donnees("BenignLight", "stateful")

# Charger et concaténer les données stateless
stateless_light_attack_data = charger_concatener_donnees("AttacksLight", "stateless")
stateless_light_benign_data = charger_concatener_donnees("BenignLight", "stateless")



# Supprimer le répertoire temporaire après avoir terminé

# Vérifier si le répertoire temporaire existe
if os.path.exists("extraction_temp"):
    # Supprimer le répertoire temporaire et son contenu
    shutil.rmtree("extraction_temp")


# Maintenant, vous avez vos données prêtes à être utilisées


In [ ]:

#### Sur-Echantillonage dataset  ####

##  Heavy Attack   72,028(stateful)      251,670 (stateless)

##  Heavy-Benign      156,014                  402,767

### Light Attack      11,295                   42,683

### Light-Benign      109,766                  281,164


#### Heavy ATTACK #########
stateful_heavy_attack_data = stateful_heavy_attack_data.sample(72028, random_state=42)
stateful_heavy_benign_data = stateful_heavy_benign_data.sample(156014, random_state=42,replace=True)

stateless_heavy_attack_data = stateless_heavy_attack_data.sample(251670, random_state=42)
stateless_heavy_benign_data = stateless_heavy_benign_data.sample(402767, random_state=42,replace=True)

#### Light ATTACK #########

# Charger les données stateful
stateful_light_attack_data = stateful_light_attack_data.sample(11295, random_state=42)
stateful_light_benign_data = stateful_light_benign_data.sample(109766, random_state=42,replace=True)
# Charger les données stateless
stateless_light_attack_data = stateless_light_attack_data.sample(42683, random_state=42)
stateless_light_benign_data = stateless_light_benign_data.sample(281164, random_state=42,replace=True)

############## taille apres re-echantillonage #############
print("Taille Stateful Heavy attack puis Stateless Heavy attack")
print(stateful_heavy_attack_data.shape)
print(stateless_heavy_attack_data.shape)

print("  \n")

print("Taille Stateful Bengin attack puis Stateless Begnin attack ")
print(stateful_heavy_benign_data.shape)
print(stateless_heavy_benign_data.shape)

print("  \n")


############## taille apres re-echantillonage #############
print("Taille Stateful Heavy Light puis Stateless Light")
print(stateful_light_attack_data.shape)
print(stateless_light_attack_data.shape)

print("  \n")

print("Taille Stateful Bengin Light puis Stateless Begnin Light")
print(stateful_light_benign_data.shape)
print(stateless_light_benign_data.shape)


#### concatenation  sur axis = 1 ########

print(" cocncatenantion sur axis = 0 sur les y \n")
print(" Heavy attack")

heavy_attack = pd.concat([stateful_heavy_attack_data, stateless_heavy_attack_data], axis=0)
print(heavy_attack.shape)

#### j'ajoute la classe / label ######

heavy_attack['class'] = 'heavy_attacks'
print(heavy_attack.shape)

print(" \n")
print(" Heavy Bengnin")

heavy_bengin = pd.concat([stateful_heavy_benign_data, stateless_heavy_benign_data], axis=0)
print(heavy_bengin.shape)

#### j'ajoute la classe / label ######

heavy_bengin['class'] = 'heavy_bengnin'
print(heavy_bengin.shape)

print(" \n")
print(" Light attack")

light_attack = pd.concat([stateful_light_attack_data, stateless_light_attack_data], axis=0)
print(light_attack.shape)


#### j'ajoute la classe / label ######
light_attack['class'] = 'light_attacks'
print(light_attack.shape)


print(" \n")
print(" Light Bengnin")

light_bengin = pd.concat([stateful_light_benign_data, stateless_light_benign_data], axis=0)
print(light_bengin.shape)

#### j'ajoute la classe / label ######
light_bengin['class'] = 'light_bengnin'
print(light_bengin.shape)



In [ ]:
################################### CONCATENATION Final des donnee sur axis = 0 ########
########################################################################################
########################################################################################


# 2. Concaténer les données

final_data = pd.concat([heavy_attack, heavy_bengin,light_attack,light_bengin], axis=0, ignore_index=True)

# 3. Supprimer les colonnes redondantes dans les données catégorielles
final_data = final_data.loc[:, ~final_data.columns.duplicated()]

# X_numerical = final_data.select_dtypes(include=['int64', 'float64'])

# X_categorical = final_data.select_dtypes(exclude='number').drop('class', axis=1)

X_numerical = final_data[['rr','A_frequency','FQDN_count','upper','lower','numeric','entropy','special', 'labels', 'labels_max','labels_average','len']]

X_categorical = final_data[['rr_type','unique_ttl','timestamp', 'longest_word', 'sld']]

y = final_data['class']

print(X_numerical.shape)
print(X_categorical.shape)

print(y.shape)
print(final_data.shape)



In [ ]:
from sklearn.impute import SimpleImputer

# Séparer les caractéristiques numériques et catégorielles

# Imputation des valeurs manquantes pour les caractéristiques numériques
numerical_imputer = SimpleImputer(strategy='mean')
X_numerical_imputed = pd.DataFrame(numerical_imputer.fit_transform(X_numerical), columns=X_numerical.columns)


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

# 2. Vérification des colonnes vides
if X_numerical.isnull().any().any():
    # Imputer les valeurs manquantes pour les caractéristiques numériques
    numerical_imputer = SimpleImputer(strategy='mean')
    X_numerical_imputed = pd.DataFrame(numerical_imputer.fit_transform(X_numerical), columns=X_numerical.columns)

    #prepocessing des features numeriques soit  avec le  LabelEncoder soit le MinMaxScaler()

    # Création d'un scaler
    scaler = MinMaxScaler()

    # Ajustement du scaler aux données
    scaler.fit(X_numerical_imputed)

    # Transformation des fonctionnalités numériques
    scaled_numeric_features = scaler.transform(X_numerical_imputed)

    # print(scaled_numeric_features)

    # Apres transformation Création d' un DataFrame à partir des valeurs transformées

    scaled_df = pd.DataFrame(scaled_numeric_features, columns=X_numerical_imputed.columns)

    # Afficher le DataFrame avec les valeurs transformées
    # print("Après transformation")
    print(scaled_df)

    total_size = scaled_df.shape
    print("Taille totale des caractéristiques numériques après transformations :", total_size)
else:
    print("Pas de valeurs manquantes dans les caractéristiques numériques. Aucune imputation nécessaire.")




In [ ]:
from sklearn.impute import SimpleImputer

# Vérification des colonnes vides
if X_categorical.isnull().any().any():
    # Imputer les valeurs manquantes pour les caractéristiques catégorielles
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    X_categorical_imputed = pd.DataFrame(categorical_imputer.fit_transform(X_categorical), columns=X_categorical.columns)

    # Afficher le DataFrame avec les valeurs imputées
    print("Après imputation")
    print(X_categorical_imputed)

    total_size_categorical = X_categorical_imputed.shape

    print("Taille totale des caractéristiques catégorielles après imputation :", total_size_categorical)

else:
    print("Pas de valeurs manquantes dans les caractéristiques catégorielles. Aucune imputation nécessaire.")


In [ ]:
from transformers import BertTokenizer

# Créer un tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased-small')

# Convertir les données catégorielles imputées en listes
categorical_features = [X_categorical_imputed[col].astype(str).tolist() for col in X_categorical_imputed.columns]

# Fusionner les textes catégoriels en une seule liste de textes
combined_texts = [' '.join([f"{col_value}" for col_value in row]) for row in zip(*categorical_features)]

# Tokeniser les textes combinés
tokens = tokenizer(combined_texts, padding=True, truncation=True, return_tensors='tf', max_length=64)

# Afficher les tokens
print(tokens)


In [ ]:
# Obtenir les longueurs de séquence
sequence_lengths = [len(token_ids) for token_ids in tokens['input_ids'].numpy()]

# Afficher l'histogramme des longueurs de séquence
plt.hist(sequence_lengths, bins=20)
plt.title('Histogramme des longueurs de séquence')
plt.xlabel('Longueur de séquence')
plt.ylabel('Fréquence')
plt.show()

In [ ]:
# Obtenir la longueur maximale
max_sequence_length = max(sequence_lengths)

# Obtenir l'indice de la plus longue séquence
indice_plus_longue_sequence = sequence_lengths.index(max_sequence_length)

# Longueur de la plus longue séquence
longueur_plus_longue_sequence = sequence_lengths[indice_plus_longue_sequence]

print(f"Longueur maximale du vecteur : {max_sequence_length}")
print(f"Longueur de la plus longue séquence : {longueur_plus_longue_sequence}")


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import  LabelEncoder

# Entrées pour l'entraînement
data_num = scaled_df
data_text = pad_sequences(tokens['input_ids'].numpy(), maxlen=max_sequence_length, padding='post')

# verification les données numériques sont correctes
print("Shape of data_num:", data_num.shape)

# verification les données textuelles sont correctes après le rembourrage
print("Shape of data_text:", data_text.shape)

# verification les étiquettes sont correctes
print("Shape of labels:", y.shape)

# Encodage des étiquettes
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(y_encoded)

In [ ]:
################# UN BERT PETIT ###########################
########## POUR EVEITER DE PLANTER MR RAM ##################


from transformers import TFBertModel, BertTokenizer
from tensorflow.keras import layers, Model
from keras_tuner.tuners import BayesianOptimization

# Entrées
input_num = layers.Input(shape=(scaled_df.shape[1],))
input_text = layers.Input(shape=(data_text.shape[1],), dtype=tf.int32)

# Branches du modèle
# Branche numérique - FFN
num_branch = layers.Dense(64, activation='relu')(input_num)
num_branch = layers.Dense(32, activation='relu')(input_num)

# Branche textuelle - BERT
bert_model = TFBertModel.from_pretrained('bert-base-uncased-small')
text_branch = bert_model(input_text)['pooler_output']

# Fusion des branches
merged = layers.concatenate([num_branch, text_branch])

# Couches supplémentaires après la fusion
merged = layers.Dropout(0.2)(merged)
merged = layers.Dense(8, activation='relu')(merged)
output = layers.Dense(4, activation='softmax')(merged)

# Création et compilation du modèle
model = Model(inputs=[input_num, input_text], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
print(model.summary())

In [ ]:
from sklearn.model_selection import train_test_split

# Diviser les données en ensembles d'entraînement et de test
text_train, text_test, num_train, num_test, labels_train, labels_test = train_test_split(
    data_text,  # Utilisation de data_text
    scaled_df,
    y_encoded,
    test_size=0.2,
    random_state=42
)


########### premier test #############################################################
############ Meme sur deux epoch j'ai toujours un probleme de ressource RAM ##########
history = model.fit([num_train, text_train], labels_train, epochs=2, batch_size=64, validation_data=([num_test, text_test], labels_test))


In [ ]:
!pip install keras-tuner


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from kerastuner.tuners import BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras import layers, Model, Input
from sklearn.metrics import accuracy_score

# Fonction pour construire le modèle
def build_model(hp):
    input_num = Input(shape=(scaled_df.shape[1],))
    input_text = Input(shape=(data_text.shape[1],), dtype=tf.int32)

    # Branches du modèle
    # Branche numérique - FFN
    num_branch = layers.Dense(hp.Choice('num_units', values=[64, 128, 256]), activation='relu')(input_num)
    num_branch = layers.Dense(hp.Choice('num_units', values=[32, 64, 128]), activation='relu')(num_branch)

    # Branche textuelle - BERT
    bert_model = TFBertModel.from_pretrained('bert-base-uncased-small')
    text_branch = bert_model(input_text)['pooler_output']

    # Fusion des branches
    merged = layers.concatenate([num_branch, text_branch])

    # Couches supplémentaires après la fusion
    merged = layers.Dropout(hp.Choice('dropout', values=[0.3, 0.5]))(merged)
    merged = layers.Dense(hp.Choice('dense_units', values=[16, 32, 64]), activation='relu')(merged)

    output = layers.Dense(4, activation='softmax')(merged)

    model = Model(inputs=[input_num, input_text], outputs=output)

    batch_size = hp.Choice('batch_size', values=[16, 32, 64])
    model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Division des données en ensembles d'entraînement et de test
text_train, text_test, num_train, num_test, labels_train, labels_test = train_test_split(
    data_text,  # Utilisation de data_text
    scaled_df,
    y_encoded,
    test_size=0.2,
    random_state=42
)

# Recherche des meilleurs hyperparamètres
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    hyperparameters=HyperParameters(),
    directory='./my_dir/',
    project_name='bert_ffn_hyperparam_tuning'
)

tuner.search([num_train, text_train], labels_train, epochs=2, validation_data=([num_test, text_test], labels_test))

# Obtention des meilleurs hyperparamètres trouvés
best_hps = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters
print(f"Meilleurs hyperparamètres: {best_hps}")


In [ ]:
best_hyperparameters = best_hps.get_config()
print("Best Hyperparameters:")
for key, value in best_hyperparameters.items():
    print(f"{key}: {value}")

In [ ]:

# Réutiliser les meilleurs hyperparamètres pour construire le modèle final
model = build_model(best_hps)

# Entraîner le modèle
history = model.fit([num_train, text_train], labels_train, epochs=40, batch_size=16, validation_data=([num_test, text_test], labels_test))


In [ ]:
from sklearn.metrics import classification_report

# Évaluation du modèle sur les données de test
evaluation_results = model.evaluate([num_test, text_test], labels_test)

# Impression des résultats d'évaluation
print("Perte sur les données de test:", evaluation_results[0])
print("Précision sur les données de test:", evaluation_results[1])

# Prédiction sur les données de test
y_pred = model.predict([num_test, text_test])

print(y_pred)

# Convertir les indices des classes prédites en étiquettes
y_pred_classes = np.argmax(y_pred, axis=1)

# Afficher le rapport de classification
class_report = classification_report(labels_test, y_pred_classes)
print("Rapport de classification :\n", class_report)


In [ ]:
from sklearn.metrics import accuracy_score

# Convertir les indices des classes prédites en étiquettes
y_true_classes = np.argmax(labels_test, axis=1) if labels_test.ndim > 1 else labels_test
y_pred_classes = np.argmax(y_pred, axis=1) if y_pred.ndim > 1 else y_pred

# Calculer l'accuracy pour chaque classe
accuracies = []
for class_label in range(4):  # Il y a 4 classes numérotées de 0 à 3
    y_true_class = (y_true_classes == class_label).astype(int)
    y_pred_class = (y_pred_classes == class_label).astype(int)
    class_accuracy = accuracy_score(y_true_class, y_pred_class)
    accuracies.append(class_accuracy)

# Afficher les accuracies pour chaque classe
for class_label, accuracy in enumerate(accuracies):
    print(f"Accuracy for class {class_label}: {accuracy}")


In [ ]:
# Extraction of training metrics
training_loss = history.history['loss']
training_accuracy = history.history['accuracy']
validation_loss = history.history['val_loss']
validation_accuracy = history.history['val_accuracy']

# Plotting loss and accuracy curves separately
epochs = range(1, len(training_loss) + 1)

# Plotting training and validation curves
plt.figure(figsize=(12, 6))

# Plotting training and validation loss
plt.subplot(1, 2, 1)
plt.plot(epochs, training_loss, label='Training loss')
plt.plot(epochs, validation_loss, label='Validation loss')
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, training_accuracy, label='Training accuracy')
plt.plot(epochs, validation_accuracy, label='Validation accuracy')
plt.title('Accuracy Curve')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Displaying both subplots
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Matrice de confusion
conf_matrix = confusion_matrix(labels_test, y_pred_classes)
class_names = ['heavy_attacks', 'heavy_benign', 'light_attacks', 'light_benign']

df_cm = pd.DataFrame(conf_matrix, index=class_names, columns=class_names)

# Afficher la matrice de confusion avec seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(df_cm, annot=True, fmt='g', cmap='Blues', cbar=False)
plt.title('Matrice de Confusion')
plt.xlabel('Prédictions')
plt.ylabel('Vraies valeurs')
plt.show()


In [ ]:
# Vérifier la forme des tableaux labels_test et y_pred_classes
print("Shape of labels_test:", labels_test.shape)
print("Shape of y_pred_classes:", y_pred_classes.shape)

from sklearn.preprocessing import LabelBinarizer

# Binariser labels_test
lb = LabelBinarizer()
labels_test_binary = lb.fit_transform(labels_test)


from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Calcul des courbes ROC et AUC pour chaque classe
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(4):
    fpr[i], tpr[i], _ = roc_curve(labels_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotter les courbes ROC pour chaque classe
plt.figure(figsize=(8, 6))
for i in range(4):
    plt.plot(fpr[i], tpr[i], lw=2, label='Class %d (AUC = %0.2f)' % (i, roc_auc[i]))

plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlabel('Taux de faux positifs (FPR)')
plt.ylabel('Taux de vrais positifs (TPR)')
plt.title('Courbes ROC pour chaque classe')
plt.legend(loc="lower right")
plt.show()
